In [3]:
from faker import Faker
import pandas as pd
import numpy as np

fake = Faker()
np.random.seed(42)

n = 30000
data = {
    "user_id": [fake.uuid4() for _ in range(n)],
    "timezone": np.random.choice(["PST", "EST", "CST"], size=n, p=[0.3, 0.4, 0.3]),
    "investment_interest": np.random.choice(["Macro", "Equities", "Crypto"], size=n),
    "email_open_prob": np.random.beta(2, 5, size=n),
    "click_through_prob": np.random.beta(2, 8, size=n),
}
df = pd.DataFrame(data)


In [4]:
pip install faker

Note: you may need to restart the kernel to use updated packages.


In [5]:
from faker import Faker
import pandas as pd
import numpy as np

fake = Faker()
np.random.seed(42)

n = 30000
data = {
    "user_id": [fake.uuid4() for _ in range(n)],
    "timezone": np.random.choice(["PST", "EST", "CST"], size=n, p=[0.3, 0.4, 0.3]),
    "investment_interest": np.random.choice(["Macro", "Equities", "Crypto"], size=n),
    "email_open_prob": np.random.beta(2, 5, size=n),
    "click_through_prob": np.random.beta(2, 8, size=n),
}
df = pd.DataFrame(data)
df.head()


,user_id,timezone,investment_interest,email_open_prob,click_through_prob
0,d4f749d7-5cb6-4715-8e1a-4f8909e44dfc,EST,Macro,0.156550,0.093785
1,8305a155-2c4f-42e9-9fa7-7f8266dabcb4,CST,Crypto,0.445872,0.286598
2,96cf7bb9-4d6a-4263-b37c-3b2be397d9d2,CST,Equities,0.245998,0.198578
3,90d6b937-c3bd-44c6-9703-bdba5f0116ed,EST,Macro,0.189009,0.451344
4,0d9652a2-b9f3-4fd0-b426-468290a627b0,PST,Equities,0.197143,0.385639


In [6]:
df["subject_line"] = np.random.choice(["A", "B"], size=n)
df["send_time"] = np.random.choice(["8AM", "9AM", "12PM"], size=n)
df["content_type"] = df["investment_interest"]  # simulate relevance


In [7]:
df["opened"] = np.random.binomial(1, df["email_open_prob"])
df["clicked"] = df.apply(
    lambda row: np.random.binomial(1, row["click_through_prob"]) if row["opened"] else 0,
    axis=1
)


In [8]:
from scipy.stats import chi2_contingency, ttest_ind

# Example: content type vs click
contingency = pd.crosstab(df["content_type"], df["clicked"])
chi2, p, _, _ = chi2_contingency(contingency)
print(f"Chi-Square p-value: {p}")


Chi-Square p-value: 0.26043115647296794


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Encode categorical features
X = df[["send_time", "subject_line", "content_type", "timezone"]]
X = pd.get_dummies(X)
y = df["clicked"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

xgb = GradientBoostingClassifier().fit(X_train, y_train)
logreg = LogisticRegression(max_iter=1000).fit(X_train, y_train)

print("XGBoost AUC:", roc_auc_score(y_test, xgb.predict_proba(X_test)[:, 1]))
print("LogReg AUC:", roc_auc_score(y_test, logreg.predict_proba(X_test)[:, 1]))


XGBoost AUC: 0.509401926986537
LogReg AUC: 0.504393448258846


In [20]:
df.to_csv("your_data.csv", index=False)
